In [1]:
import urllib2
import numpy as np
import json
import pandas as pd
pd.set_option("display.max_columns",999)
from pandas.io.json import json_normalize
import os
import pickle

In [2]:
#number per page = 30 
total_records = 100
per_page = 30
n=int(total_records/per_page)
alldata =[]    
#loop to get all pages (n) (API only allows a max of 100 enteries per page)
for i in xrange(0, n):
    try:
        print "Downloading Page {}".format(i)
        api ='http://api.inaturalist.org/v1/observations?page={}&quality_grade=any&identifications=any&captive=false&iconic_taxa%5B%5D=Protozoa&iconic_taxa%5B%5D=Animalia&iconic_taxa%5B%5D=Mollusca&iconic_taxa%5B%5D=Actinopterygii&iconic_taxa%5B%5D=Amphibia&iconic_taxa%5B%5D=Aves&iconic_taxa%5B%5D=Mammalia&iconic_taxa%5B%5D=Reptilia&iconic_taxa%5B%5D=Arachnida&iconic_taxa%5B%5D=Insecta&iconic_taxa%5B%5D=Plantae&iconic_taxa%5B%5D=Fungi&iconic_taxa%5B%5D=Chromista&iconic_taxa%5B%5D=unknown&place_id=14'.format(i)
        req = urllib2.Request(api)
        response = urllib2.urlopen(req)
        response_read = response.read()
        json_data=json.loads(response_read)
        df = json_normalize(json_data, 'results', meta = ['page','per_page','total_results'])
        alldata.append(df)
    except:
        pass  
        

In [ ]:
#pickle.dump(alldata, open("inat_pickle.p", "wb"))

In [3]:
iNat_df = pd.concat(alldata)

In [4]:
iNat_df.shape

(60, 68)

In [12]:
iNat_df.taxon.to_csv("test.csv")

In [ ]:
iNat_df.

In [ ]:
features_to_keep = ['cached_votes_total','captive','comments_count','community_taxon_id','created_at',
                    'faves_count','geojson','id_please',
                    'identifications_count','identifications_most_agree','identifications_most_disagree',
                    'identifications_some_agree','location','mappable','obscured','observed_on',
                    'observed_on_details','observed_on_string','owners_identification_from_vision','place_ids','taxon']
#features_to_keep = ['cached_votes_total','captive']
iNat_df_main = iNat_df[features_to_keep]

In [ ]:
iNat_df_main.head()

In [ ]:
iNat_df_main.info()

In [ ]:
def flattenColumn(df, column):
    flat_columns = df[column].apply(pd.Series)
    flat_columns.columns = [column+'_'+str(flat_column_name) for flat_column_name in list(flat_columns.columns)]
    new_df= df.drop(column, 1)
    new_df= pd.concat([new_df,flat_columns], axis =1)
    return new_df

iNat_df_main = flattenColumn(iNat_df_main, 'taxon')
#iNat_df_main = flattenColumn(iNat_df_main, 'place_ids')
#iNat_df_main = flattenColumn(iNat_df_main, 'geojson')
#iNat_df_main = flattenColumn(iNat_df_main, 'identifications')

In [ ]:
#iNat_df_main = flattenColumn(iNat_df_main, 'taxon_establishment_means')

In [ ]:
iNat_df_main = flattenColumn(iNat_df_main, 'geojson')

In [ ]:
iNat_df_main = flattenColumn(iNat_df_main, 'observed_on_details')

In [ ]:
iNat_df_main.created_at = pd.to_datetime(iNat_df_main.created_at,infer_datetime_format=True)

In [ ]:
iNat_df_main.created_at.iloc[0]

In [ ]:
iNat_df_main.location= iNat_df_main.location.apply(lambda x: str(x))

In [ ]:
iNat_df_main.observed_on= iNat_df_main.observed_on.apply(lambda x: str(x))

In [ ]:
iNat_df_main.observed_on_string = pd.to_datetime(iNat_df_main.observed_on_string,infer_datetime_format=True)

In [ ]:
iNat_df_main.taxon_created_at = pd.to_datetime(iNat_df_main.taxon_created_at,infer_datetime_format=True)

In [ ]:
cols_to_drop = ['taxon_default_photo','taxon_min_species_ancestry',
                'taxon_native','taxon_preferred_establishment_means',
                'taxon_establishment_means']

In [ ]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [ ]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [ ]:
sparkdf = sqlsc.createDataFrame(iNat_df_main)

In [ ]:
sparkdf.printSchema()

In [ ]:
#sparkdf.groupby(data['species_guess']).max().show()
sparkdf.registerTempTable("results")


In [ ]:
sqlsc.sql("select distinct quality_grade from results").show()